*Copyright (C) 2023 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Serialization of Lava networks

A Lava network can be saved to and later loaded from a file, this is also called serialization. Serialization of Lava networks, a list of Lava processes which are optionally connected and/or compiled, is based on the pickle module from python. The pickle module implements binary protocols for serializing and de-serializing a Python object structure.


<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_images/lava_notebooks_in_depth_tutorial11_serialization.png" width="600" align="center"/>

Serialization of compiled Lava networks allows to easily share bigger Lava networks and to execute pre-compiled networks. This enables to run networks which take a long time to compile faster, which decreases the time to execution. Pre-compilation also enables the execution of complex networks on embedded environments, which otherwise would suffer from long compilation times.

> **⚠️ Warning**  
> Using `save` and `load` is not secure. Only load data you trust.
> As it uses pickle it is possible to construct malicious data which will execute arbitrary code during loading. Never load data that could have come from an untrusted source, or that could have been tampered with.
>
> Consider signing data with `hmac` if you need to ensure that it has not been tampered with.

## Interface

A list of configured Lava processes can be stored in a file by calling `save`. The arguments are a Lava process or a list of Lava processes, a string defining the path and name of the file and a Executable object. The Executable object is needed to store compiled Lava networks.

```
save(processes, filename, executable=None)
```
As default file extension `.pickle` is used, if no other file extension is provided in the argument `filename`.

Stored Lava processes and their corresponding executable can be retrieved by calling `load`. The argument is a string defining the path and name of the file. The function returns the list of stored Lava processes and the corresponding executable.
```
processes, executable = load(filename)
```

## Example

In [17]:
import numpy as np
import tempfile
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.magma.core.run_configs import Loihi2SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.utils.serialization import save, load

A simple LIF->Dense->LIF network is configured.

In [18]:
pre_size = 2
post_size = 3

# Create network
weights = np.ones((post_size, pre_size))

lif_in = LIF(shape=(pre_size,), bias_mant=100, vth=120,
                    name="LIF_neuron input")
dense = Dense(weights=weights * 10, name="Dense")
lif_out = LIF(shape=(post_size,), bias_mant=0, vth=50000,
                    name="LIF_neuron output")

# Connect the processes
lif_in.s_out.connect(dense.s_in)
dense.a_out.connect(lif_out.a_in)

In order to store a compiled version of the Lava network an `executable` is needed. Calling `compile` on a Lava process compiles the process and all its connected processes and returns an `executable`.

In [19]:
# Create executable
ex = lif_in.compile(run_cfg=Loihi2SimCfg())

Storing all Lava processes and the corresponding executable in a file using `save`.
> **ℹ️ Note**  
> The context manager `TemporaryDirectory` is used to prevent write permission errors and ensure the clean up of created files from this tutorial. The context manager is not necessary or recommended when using `save` and `load`.

Afterwords, the list of processes and their corresponding executable are retrieved from the file using `load`.

In [20]:
# Context manager is used only to ensure clean up
with tempfile.TemporaryDirectory() as tmpdirname:
    # Store the Lava network
    save(processes=[lif_in, lif_out, dense],
         filename=tmpdirname + "test",
         executable=ex)

    # Load executable again
    procs, loaded_ex = load(tmpdirname + "test.pickle")

The list of processes was assigned to `procs`. These process instances are not equal to the instances created initially, although they contain the same information and behave the same way. Also the loaded executable is not equal to the initially created executable.

Looking at the list of processes shows which processes have been loaded from the file. To use the processes again they can be assigned to new variables.

In [21]:
print(procs)
input_lif = procs[0]
output_lif = procs[1]

[<lava.proc.lif.process.LIF object at 0x7fc461a4ff40>, <lava.proc.lif.process.LIF object at 0x7fc461abf370>, <lava.proc.dense.process.Dense object at 0x7fc461abfc40>]


Using the loaded `executable` a runtime needs to be created in order to execute the compiled Lava network. If `create_runtime` is not used, then the processes will be compiled again when calling `run`. For pre-compiled networks this should be avoided.

In [22]:
# Create the runtime from the loaded executable
input_lif.create_runtime(executable=loaded_ex)

Finally, the network is executed and the voltage and current of the input and output LIF neurons are stored.

In [23]:
num_steps = 5
output_lif_in_v = np.zeros(shape=(pre_size, num_steps))
output_lif_out_u = np.zeros(shape=(post_size, num_steps))

# Execute the network and retrieve voltage and current
try:
    for i in range(num_steps):
        input_lif.run(condition=RunSteps(num_steps=1))

        output_lif_in_v[:, i] = input_lif.v.get()
        output_lif_out_u[:, i] = output_lif.u.get()
finally:
    input_lif.stop()

Showing the results.

In [24]:
print("Voltage of 2 input LIF neurons over time steps: ")
print(output_lif_in_v)
print("Current of 3 output LIF neurons over time steps: ")
print(output_lif_out_u)

Voltage of 2 input LIF neurons over time steps: 
[[100.   0. 100.   0. 100.]
 [100.   0. 100.   0. 100.]]
Current of 3 output LIF neurons over time steps: 
[[ 0.  0. 20. 20. 40.]
 [ 0.  0. 20. 20. 40.]
 [ 0.  0. 20. 20. 40.]]


A hierarchical process can also be stored to a file and all used sub-processes will be available after retrieving it again from a file. This is helpful to create a pre-compiled network, which can be accessed and executed through a single variable.

## How to learn more?

If you want to find out more about Lava, have a look at the [Lava documentation](https://lava-nc.org).

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to the [INRC newsletter](http://eepurl.com/hJCyhb).